In [ ]:
using CSV
using DataFrames 
using Pipe

In [ ]:
using MLJ
using OutlierDetection
using OutlierDetectionNeighbors: KNNDetector
using OutlierDetectionData: ODDS

In [ ]:
ENV["LINES"], ENV["COLUMNS"] = 15, 200

In [77]:
ls_loc = "../dataset/dataset/last_year/"

"../dataset/dataset/last_year/"

In [109]:
exc = ["Column1", "土地位置建物門牌", "編號", "非都市土地使用分區", "非都市土地使用編定", 
    "交易年月日", "建築完成年月", "備註", "移轉編號", "period"]
println("")

In [113]:
ls_tp = @pipe CSV.File(ls_loc*"LastYear_tp.csv") |> 
    DataFrame |>
    select(_, Not(exc))
println("")

In [114]:
ls_ks = @pipe CSV.File(ls_loc*"LastYear_ks.csv") |> 
    DataFrame |>
    select(_, Not(exc))
println("")

In [115]:
ls_nt = @pipe CSV.File(ls_loc*"LastYear_nt.csv")  |> 
    DataFrame |>
    select(_, Not(exc))
println("")

In [116]:
ls_tc = @pipe CSV.File(ls_loc*"LastYear_tc.csv") |> 
    DataFrame |>
    select(_, Not(exc))
println("")

In [117]:
ls_tn = @pipe CSV.File(ls_loc*"LastYear_tn.csv")  |> 
    DataFrame |>
    select(_, Not(exc))
println("")

In [118]:
ls_ty = @pipe CSV.File(ls_loc*"LastYear_ty.csv") |> 
    DataFrame |>
    select(_, Not(exc))
println("")